# Predicting All-NBA Team and Player Salaries - Data Acquisition
---

This notebook launches our project by webscraping National Basketball Association (NBA) player and team data on salary and statistics (total, per-game, and advanced) from various basketball sites. Most of this data will come from popular statistics website ```Basketball-Reference```, while others is also sourced from ```HoopsHype``` and ```Wikipedia```. Additional data from Kaggle datasets may be used at a later time as needed.

Using webscraping methods ```BeautifulSoup``` and ```Selenium```, we will start by saving a snapshot of the HTML page housing our desired statistics. We will then use those pages to inspect and pull the tables the interest into CSV's for further data cleaning and feature engineering. You will see that throughout this notebook we incorporate sleep times / rate limits between data pulling requests, this will prevent server overload and potential IP blocking, and will ensure respectful and responsible webscraping. 

See the end of this notebook for a list of [sources](#Special-Thanks-to-These-Sources) and tutorials used in developing my webscraping script. 

Further detailed notebooks on the various segments of this project can be found at the following: 
- [02_Data_Cleaning](./02_Data_Cleaning.ipynb)
- [03_Preliminary_EDA](./03_Preliminary_EDA.ipynb)
- [04_Data_Modeling_I](./04_Data_Modeling_I.ipynb)
- [05_Data_Modeling_II](./05_Data_Modeling_II.ipynb)

For more information on the background, a summary of methods, and findings, please see the associated [README](../README.md) for this analysis. 

## Contents
- [Preliminaries](#Preliminaries)
    - [Define Year of Interest](#Define-Years-of-Interest)
    - [Selenium Setup](#Selenium-Setup)
- [1. Salary Data](#1.-Salary-Data)
    - [I. Player Salary](#I.-Player-Salary)
    - [II. Team Payroll](#II.-Team-Payroll)
    - [III. Salary Caps](#III.-Salary-Caps)
- [2. Statistics Data](#2.-Statistics-Data)
    - [IV. Player Statistics](#IV.-Player-Statistics)
    - [V. All-NBA Team Winners and Nominees](#V.-All-NBA-Team-Winners-and-Nominees)
- [3. Additional Performance-Related Data](#3.-Additional-Performance-Related-Data)
    - [VI. Team Rankings](#VI.-Team-Rankings)
    - [VII. Team Standings](#VII.-Team-Standings)
    - [VIII. All-Star-Appearance](#VIII.-All-Star-Appearance)
- [4. Misc Data](#4.-Misc-Data)
    - [IV. NBA Draft Year](#IV.-NBA-Draft-Year)
---

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [2]:
# pip install selenium

# Preliminaries

### Define Years of Interest
##### To be periodically updated to include latest available seasons. This will enable the model to remain robust and reflective of the most recent trends and patterns in the NBA.

Note: Year variable may need to be manipulated by adding +1 when called to mirror which season the specific site is referring to. We are considering each year to refer to the the season _start_ date (e.g., 1991 will refer to the 1991-1992 season). Some sites, such as that showing team rankings, understandably will refer to the 1991 team ranking based on the season _end_ date (e.g., 1991 will refer to the 1990-1991 season). We will adjust the year variable accordingly in our loops.

In [3]:
years = list(range(1990,2023)) # stopping at 2022 which will be the 2022-23 season

### Selenium Setup

##### Several of the following sections ([Player Statistics](#V.-Player-Statistics), [Salary Caps](#VI.-Salary-Caps), [All-NBA Team Winners](#VII.-All-NBA-Team-Winners-and-Nominees)) will use the [Selenium](https://selenium-python.readthedocs.io/index.html) Python package to webscrape dynamic tables which render all rows using JavaScript after the page has loaded. To ensure smooth execution for other users, follow these steps if you wish to run the provided code:

1. Install Selenium with ```pip install selenium```
<br></br>
2. Choose the web browser you intend to run this code on (different browsers require different drivers) and ensure that you have the latest version of the browser installed.
<br></br>
3. Download, extract, and save [Selenium webdriver](https://selenium-python.readthedocs.io/installation.html#drivers) for your chosen browser to the current ```code``` folder housing this notebook.
    1. Perform the download, extract, save
    2. Operationalize webdriver
        1. <u>Method 1</u>: Specify location of driver in the cell below, for example: ```driver = webdriver.Edge(executable_path='c:\User-Path\msedgedriver.exe')```. Replace the following:
            - ```Edge``` with the browser you installed the driver for, 
            - ```User-Path``` with the path where the driver is saved, and 
            - ```msedgedriver.exe```  with the name of the extracted driver
        2. <u>Method 2</u>: To **bypass** having to perform Method 1 with each use of the webdriver (and at all), consider adding the extracted webdriver to the Path variable on your machine (accessible via Advanced System Settings on your computer). In this case, do not update the ```driver = webdriver.Edge()``` code below, except for changing ```Edge``` to your preferred browser.
<br></br>
4. As new versions of browsers/webdrivers become available, **re-perform** Steps 2 & 3 above.

By following these steps, you'll ensure that the provided code continues to work smoothly as you access the latest player statistics using Selenium.

In [4]:
driver = webdriver.Edge()

---

## **1. Salary Data**

## I. Player Salary

In [5]:
for year in years:
    url = f'https://hoopshype.com/salaries/players/{year}-{year+1}/'
    res = requests.get(url)
    if res.status_code >= 200:
        with open(f"webscraping/salary/{year}.html", "w+", encoding="utf-8") as file:
            file.write(res.text)
            lag = np.random.uniform(4,6)
            print(f'Finished writing {year}, waiting ... {round(lag,2)}')
            time.sleep(lag)

Finished writing 1990, waiting ... 5.16
Finished writing 1991, waiting ... 4.81
Finished writing 1992, waiting ... 4.36
Finished writing 1993, waiting ... 4.59
Finished writing 1994, waiting ... 4.24
Finished writing 1995, waiting ... 4.46
Finished writing 1996, waiting ... 4.75
Finished writing 1997, waiting ... 5.78
Finished writing 1998, waiting ... 5.47
Finished writing 1999, waiting ... 5.49
Finished writing 2000, waiting ... 5.07
Finished writing 2001, waiting ... 4.42
Finished writing 2002, waiting ... 4.96
Finished writing 2003, waiting ... 4.54
Finished writing 2004, waiting ... 5.49
Finished writing 2005, waiting ... 5.08
Finished writing 2006, waiting ... 5.71
Finished writing 2007, waiting ... 5.92
Finished writing 2008, waiting ... 5.91
Finished writing 2009, waiting ... 4.16
Finished writing 2010, waiting ... 5.67
Finished writing 2011, waiting ... 5.03
Finished writing 2012, waiting ... 4.74
Finished writing 2013, waiting ... 4.61
Finished writing 2014, waiting ... 5.67


In [6]:
sal_all = []

for year in years:
    with open(f'webscraping/salary/{year}.html', encoding="utf-8") as file:
        page = file.read()
    soup = BeautifulSoup(page, 'html.parser')
    salary_table = soup.find(class_="hh-salaries-ranking-table")
    salaries1 = pd.read_html(str(salary_table))[0].drop(columns="Unnamed: 0")    
    salaries1['Year'] = year
    salaries1.rename(columns={salaries1.columns.tolist()[1]: 'Salary',
                              salaries1.columns.tolist()[2]: 'Salary_Adj'}, inplace=True)
    
    print(f'Finished scraping {year}')
    sal_all.append(salaries1)

Finished scraping 1990
Finished scraping 1991
Finished scraping 1992
Finished scraping 1993
Finished scraping 1994
Finished scraping 1995
Finished scraping 1996
Finished scraping 1997
Finished scraping 1998
Finished scraping 1999
Finished scraping 2000
Finished scraping 2001
Finished scraping 2002
Finished scraping 2003
Finished scraping 2004
Finished scraping 2005
Finished scraping 2006
Finished scraping 2007
Finished scraping 2008
Finished scraping 2009
Finished scraping 2010
Finished scraping 2011
Finished scraping 2012
Finished scraping 2013
Finished scraping 2014
Finished scraping 2015
Finished scraping 2016
Finished scraping 2017
Finished scraping 2018
Finished scraping 2019
Finished scraping 2020
Finished scraping 2021
Finished scraping 2022


In [7]:
salaries = pd.concat(sal_all)
salaries.to_csv('../data/salaries.csv', index=False)
print(salaries.shape)
print(f'{salaries.Year.min()}-{salaries.Year.max()}')

(15778, 4)
1990-2022


## II. Team Payroll

In [8]:
for year in years:
    url = f'https://hoopshype.com/salaries/{year}-{year+1}/'
    res = requests.get(url)
    if res.status_code >= 200:
        with open(f"webscraping/team/payroll/{year}.html", "w+", encoding="utf-8") as file:
            file.write(res.text)
            lag = np.random.uniform(4,6)
            print(f'Finished writing {year}, waiting ... {round(lag,2)}')
            time.sleep(lag)

Finished writing 1990, waiting ... 5.73
Finished writing 1991, waiting ... 5.31
Finished writing 1992, waiting ... 5.38
Finished writing 1993, waiting ... 4.86
Finished writing 1994, waiting ... 4.61
Finished writing 1995, waiting ... 5.0
Finished writing 1996, waiting ... 5.34
Finished writing 1997, waiting ... 4.32
Finished writing 1998, waiting ... 5.41
Finished writing 1999, waiting ... 4.33
Finished writing 2000, waiting ... 4.74
Finished writing 2001, waiting ... 5.15
Finished writing 2002, waiting ... 5.15
Finished writing 2003, waiting ... 5.48
Finished writing 2004, waiting ... 4.69
Finished writing 2005, waiting ... 4.75
Finished writing 2006, waiting ... 5.8
Finished writing 2007, waiting ... 4.83
Finished writing 2008, waiting ... 4.73
Finished writing 2009, waiting ... 4.97
Finished writing 2010, waiting ... 5.55
Finished writing 2011, waiting ... 4.66
Finished writing 2012, waiting ... 5.46
Finished writing 2013, waiting ... 4.81
Finished writing 2014, waiting ... 5.66
Fi

In [9]:
teamsal_all = []

for year in years:
    with open(f'webscraping/team/payroll/{year}.html', encoding="utf-8") as file:
        page = file.read()
    soup = BeautifulSoup(page, 'html.parser')
    teamsalary_table = soup.find(class_="hh-salaries-ranking-table")
    teamsalaries1 = pd.read_html(str(teamsalary_table))[0].drop(columns="Unnamed: 0")    
    teamsalaries1['Year'] = year
    teamsalaries1.rename(columns={teamsalaries1.columns.tolist()[1]: 'Payroll',
                              teamsalaries1.columns.tolist()[2]: 'Payroll_Adj'}, inplace=True) # Adjusted for 2022-2023 dollars
    
    print(f'Finished scraping {year}')
    teamsal_all.append(teamsalaries1)    

Finished scraping 1990
Finished scraping 1991
Finished scraping 1992
Finished scraping 1993
Finished scraping 1994
Finished scraping 1995
Finished scraping 1996
Finished scraping 1997
Finished scraping 1998
Finished scraping 1999
Finished scraping 2000
Finished scraping 2001
Finished scraping 2002
Finished scraping 2003
Finished scraping 2004
Finished scraping 2005
Finished scraping 2006
Finished scraping 2007
Finished scraping 2008
Finished scraping 2009
Finished scraping 2010
Finished scraping 2011
Finished scraping 2012
Finished scraping 2013
Finished scraping 2014
Finished scraping 2015
Finished scraping 2016
Finished scraping 2017
Finished scraping 2018
Finished scraping 2019
Finished scraping 2020
Finished scraping 2021
Finished scraping 2022


In [10]:
teamsalaries = pd.concat(teamsal_all)
teamsalaries.to_csv('../data/team_payroll.csv', index=False)
print(teamsalaries.shape)
print(f'{teamsalaries.Year.min()}-{teamsalaries.Year.max()}')

(966, 4)
1990-2022


## III. Salary Caps

In [11]:
url = 'https://www.basketball-reference.com/contracts/salary-cap-history.html'
driver.get(url)
driver.execute_script('window.scrollTo(1,10000)')
html = driver.page_source

with open('webscraping/salary/salarycap/salarycap.html', "w+", encoding="utf-8") as file:
    file.write(html)

In [12]:
with open('webscraping/salary/salarycap/salarycap.html', encoding="utf-8") as file:
    page = file.read()
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', {'id' : 'salary_cap_history'})
salarycap = pd.read_html(str(table))[0] 

In [13]:
salarycap
salarycap.to_csv('../data/salarycap.csv', index=False)

---

## **2. Statistics Data**

## IV. Player Statistics
##### We will be scraping 3 types of player statistics: per-game, totals, and advanced stats.

In [14]:
stats = ['per_game', 'totals', 'advanced']

In [15]:
for stat in stats:
    for year in years:
        url = f'https://www.basketball-reference.com/leagues/NBA_{year+1}_{stat}.html' # +1 here because site is determining year by the season end date and our list is defining it by season start
        driver.get(url)
        driver.execute_script('window.scrollTo(1,10000)')
        html = driver.page_source

        with open(f'webscraping/players/{stat}/{year}.html', "w+", encoding="utf-8") as file:
            file.write(html)
        lag = np.random.uniform(4,6)
        print(f'Finished writing {year} {stat} stats, waiting ... {round(lag,2)}')
        time.sleep(lag)

Finished writing 1990 per_game stats, waiting ... 5.97
Finished writing 1991 per_game stats, waiting ... 4.72
Finished writing 1992 per_game stats, waiting ... 5.29
Finished writing 1993 per_game stats, waiting ... 5.59
Finished writing 1994 per_game stats, waiting ... 5.09
Finished writing 1995 per_game stats, waiting ... 5.8
Finished writing 1996 per_game stats, waiting ... 5.16
Finished writing 1997 per_game stats, waiting ... 4.58
Finished writing 1998 per_game stats, waiting ... 5.79
Finished writing 1999 per_game stats, waiting ... 5.77
Finished writing 2000 per_game stats, waiting ... 4.83
Finished writing 2001 per_game stats, waiting ... 4.11
Finished writing 2002 per_game stats, waiting ... 5.35
Finished writing 2003 per_game stats, waiting ... 4.13
Finished writing 2004 per_game stats, waiting ... 4.62
Finished writing 2005 per_game stats, waiting ... 4.06
Finished writing 2006 per_game stats, waiting ... 4.69
Finished writing 2007 per_game stats, waiting ... 5.08
Finished wr

In [16]:
pg_all = []
tot_all = []
adv_all = []

for stat in stats:
    for year in years:  
        with open(f'webscraping/players/{stat}/{year}.html', encoding="utf-8") as file:
            page = file.read()
        soup = BeautifulSoup(page, 'html.parser')
        table = soup.find('table', {'id' : f'{stat}_stats'})
        soup.find('tr', class_ = 'thead').decompose()
        df = pd.read_html(str(table))[0] 
        df['Year'] = year
        df['Stat'] = stat
        
        print(f'Finished scraping {year} {stat} stats')
        
        if stat == 'per_game':
            pg_all.append(df)
        if stat == 'totals':
            tot_all.append(df)
        if stat == 'advanced':
            adv_all.append(df)

Finished scraping 1990 per_game stats
Finished scraping 1991 per_game stats
Finished scraping 1992 per_game stats
Finished scraping 1993 per_game stats
Finished scraping 1994 per_game stats
Finished scraping 1995 per_game stats
Finished scraping 1996 per_game stats
Finished scraping 1997 per_game stats
Finished scraping 1998 per_game stats
Finished scraping 1999 per_game stats
Finished scraping 2000 per_game stats
Finished scraping 2001 per_game stats
Finished scraping 2002 per_game stats
Finished scraping 2003 per_game stats
Finished scraping 2004 per_game stats
Finished scraping 2005 per_game stats
Finished scraping 2006 per_game stats
Finished scraping 2007 per_game stats
Finished scraping 2008 per_game stats
Finished scraping 2009 per_game stats
Finished scraping 2010 per_game stats
Finished scraping 2011 per_game stats
Finished scraping 2012 per_game stats
Finished scraping 2013 per_game stats
Finished scraping 2014 per_game stats
Finished scraping 2015 per_game stats
Finished scr

In [17]:
b0 ='\033[1m'
b1 = '\033[0;0m'

In [18]:
pg = pd.concat(pg_all)
pg.to_csv('../data/per_game_data.csv', index=False)
print(f' --- {b0}Per Game{b1} ---')
print(pg.shape)
print(f'{pg.Year.min()}-{pg.Year.max()}')

tot = pd.concat(tot_all)
tot.to_csv('../data/totals_data.csv', index=False)
print(f'\n --- {b0}Totals{b1} ---')
print(tot.shape)
print(f'{tot.Year.min()}-{tot.Year.max()}')

adv = pd.concat(adv_all)
adv.to_csv('../data/advanced_data.csv', index=False)
print(f'\n --- {b0}Advanced{b1} ---')
print(adv.shape)
print(f'{adv.Year.min()}-{adv.Year.max()}')

 --- Per Game ---
(19589, 32)
1990-2022

 --- Totals ---
(19589, 32)
1990-2022

 --- Advanced ---
(19589, 31)
1990-2022


In [19]:
# PREVIEW TABLES
pg

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Stat
0,1,Alaa Abdelnaby,PF,22,POR,43,0,6.7,1.3,2.7,...,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990,per_game
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19,22.5,6.2,15.1,...,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1990,per_game
2,3,Mark Acres,C,28,ORL,68,0,19.3,1.6,3.1,...,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1990,per_game
3,4,Michael Adams,PG,28,DEN,66,66,35.5,8.5,21.5,...,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1990,per_game
4,5,Mark Aguirre,SF,31,DET,78,13,25.7,5.4,11.7,...,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1990,per_game
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,535,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,...,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2022,per_game
700,536,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,...,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2022,per_game
701,537,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,...,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2022,per_game
702,538,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,...,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,2022,per_game


In [20]:
tot

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Stat
0,1,Alaa Abdelnaby,PF,22,POR,43,0,290,55,116,...,62,89,12,4,12,22,39,135,1990,totals
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19,1505,417,1009,...,87,121,206,55,4,110,149,942,1990,totals
2,3,Mark Acres,C,28,ORL,68,0,1313,109,214,...,219,359,25,25,25,42,218,285,1990,totals
3,4,Michael Adams,PG,28,DEN,66,66,2346,560,1421,...,198,256,693,147,6,240,162,1752,1990,totals
4,5,Mark Aguirre,SF,31,DET,78,13,2006,420,909,...,240,374,139,47,20,128,209,1104,1990,totals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,535,Thaddeus Young,PF,34,TOR,54,9,795,108,198,...,95,166,75,54,5,42,88,240,2022,totals
700,536,Trae Young,PG,24,ATL,73,73,2541,597,1390,...,161,217,741,80,9,300,104,1914,2022,totals
701,537,Omer Yurtseven,C,24,MIA,9,0,83,16,27,...,15,23,2,2,2,4,16,40,2022,totals
702,538,Cody Zeller,C,30,MIA,15,2,217,37,59,...,39,64,10,3,4,14,33,98,2022,totals


In [21]:
adv

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Year,Stat
0,1,Alaa Abdelnaby,PF,22,POR,43,290,13.1,.499,.000,...,0.5,0.5,.079,NaN,-3.4,-1.2,-4.6,-0.2,1990,advanced
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,1505,12.2,.448,.099,...,-0.3,-1.0,-0.031,NaN,-2.0,-3.0,-5.0,-1.1,1990,advanced
2,3,Mark Acres,C,28,ORL,68,1313,9.2,.551,.014,...,1.1,2.5,.090,NaN,-2.8,-0.2,-3.0,-0.3,1990,advanced
3,4,Michael Adams,PG,28,DEN,66,2346,22.3,.530,.397,...,0.4,6.3,.128,NaN,6.0,-0.7,5.3,4.3,1990,advanced
4,5,Mark Aguirre,SF,31,DET,78,2006,16.7,.526,.086,...,2.7,5.5,.132,NaN,1.2,0.2,1.4,1.7,1990,advanced
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,535,Thaddeus Young,PF,34,TOR,54,795,14.1,.573,.172,...,1.1,1.8,.109,NaN,-1.8,1.9,0.1,0.4,2022,advanced
700,536,Trae Young,PG,24,ATL,73,2541,22.0,.573,.331,...,1.4,6.7,.126,NaN,5.3,-2.0,3.3,3.4,2022,advanced
701,537,Omer Yurtseven,C,24,MIA,9,83,16.7,.675,.259,...,0.1,0.3,.159,NaN,-2.5,-1.5,-3.9,0.0,2022,advanced
702,538,Cody Zeller,C,30,MIA,15,217,16.4,.659,.034,...,0.3,0.7,.147,NaN,-2.0,-0.7,-2.8,0.0,2022,advanced


## V. All-NBA Team Winners and Nominees

In [22]:
for year in years:
    url = f'https://www.basketball-reference.com/awards/awards_{year+1}.html'
    driver.get(url)
    driver.execute_script('window.scrollTo(1,10000)') # Assures entire page is saved for scraping
    html = driver.page_source

    with open(f'webscraping/players/all_team/{year}.html', "w+", encoding="utf-8") as file:
        file.write(html)
    lag = np.random.uniform(4,6)
    print(f'Finished writing {year}, waiting ... {round(lag,2)}')
    time.sleep(lag)

Finished writing 1990, waiting ... 4.89
Finished writing 1991, waiting ... 5.12
Finished writing 1992, waiting ... 5.11
Finished writing 1993, waiting ... 5.8
Finished writing 1994, waiting ... 5.85
Finished writing 1995, waiting ... 4.66
Finished writing 1996, waiting ... 4.91
Finished writing 1997, waiting ... 4.15
Finished writing 1998, waiting ... 4.95
Finished writing 1999, waiting ... 5.24
Finished writing 2000, waiting ... 5.59
Finished writing 2001, waiting ... 4.21
Finished writing 2002, waiting ... 6.0
Finished writing 2003, waiting ... 4.51
Finished writing 2004, waiting ... 5.83
Finished writing 2005, waiting ... 4.48
Finished writing 2006, waiting ... 4.75
Finished writing 2007, waiting ... 4.54
Finished writing 2008, waiting ... 5.89
Finished writing 2009, waiting ... 5.64
Finished writing 2010, waiting ... 5.7
Finished writing 2011, waiting ... 4.15
Finished writing 2012, waiting ... 4.66
Finished writing 2013, waiting ... 4.52
Finished writing 2014, waiting ... 4.93
Fin

In [23]:
all = []

for year in years:  
    with open(f'webscraping/players/all_team/{year}.html', encoding="utf-8") as file:
        page = file.read()
    soup = BeautifulSoup(page, 'html.parser')
    table = soup.find('table', {'id' : 'leading_all_nba'})
    try:
        soup.find('tr', class_ = 'over_header').decompose()
        soup.find('tr', {'id' : 'start_2nd'}).decompose()
        soup.find('tr', {'id' : 'start_3rd'}).decompose()
        soup.find('tr', {'id' : 'start_ORV'}).decompose()
        soup.find('div', class_ = 'topscroll_div').decompose()
    except:
        soup.find('tr', class_ = 'over_header').decompose()
        soup.find('tr', {'id' : 'start_1T'}).decompose()
        soup.find('tr', {'id' : 'start_2T'}).decompose()
        soup.find('tr', {'id' : 'start_3T'}).decompose()
        soup.find('tr', {'id' : 'start_ORV'}).decompose()
        soup.find('div', class_ = 'topscroll_div').decompose()
               
    df = pd.read_html(str(table), header=1)[0] 
    df['Year'] = year   
    
    print(f'Finished scraping {year}')
    all.append(df)    

Finished scraping 1990
Finished scraping 1991
Finished scraping 1992
Finished scraping 1993
Finished scraping 1994
Finished scraping 1995
Finished scraping 1996
Finished scraping 1997
Finished scraping 1998
Finished scraping 1999
Finished scraping 2000
Finished scraping 2001
Finished scraping 2002
Finished scraping 2003
Finished scraping 2004
Finished scraping 2005
Finished scraping 2006
Finished scraping 2007
Finished scraping 2008
Finished scraping 2009
Finished scraping 2010
Finished scraping 2011
Finished scraping 2012
Finished scraping 2013
Finished scraping 2014
Finished scraping 2015
Finished scraping 2016
Finished scraping 2017
Finished scraping 2018
Finished scraping 2019
Finished scraping 2020
Finished scraping 2021
Finished scraping 2022


In [24]:
all_nba = pd.concat(all)
all_nba.to_csv('../data/all_nba_teams.csv', index=False)
print(all_nba.shape)
print(f'{all_nba.Year.min()}-{all_nba.Year.max()}')

(1354, 24)
1990-2022


---

## **3. Additional Performance-Related Data**

## VI. Team Rankings

In [25]:
for year in years:
    url = f'https://www.basketball-reference.com/leagues/NBA_{year+1}_ratings.html' # +1 here because site is determining year by the season end date and our list is defining it by season start
    res = requests.get(url)
    if res.status_code >= 200:
        with open(f"webscraping/team/rank/{year}.html", "w+", encoding="utf-8") as file:
            file.write(res.text)
            lag = np.random.uniform(4,6)
            print(f'Finished writing {year}, waiting ... {round(lag,2)}')
            time.sleep(lag)

Finished writing 1990, waiting ... 4.14
Finished writing 1991, waiting ... 5.74
Finished writing 1992, waiting ... 4.78
Finished writing 1993, waiting ... 5.97
Finished writing 1994, waiting ... 5.78
Finished writing 1995, waiting ... 4.45
Finished writing 1996, waiting ... 4.64
Finished writing 1997, waiting ... 4.1
Finished writing 1998, waiting ... 4.97
Finished writing 1999, waiting ... 5.55
Finished writing 2000, waiting ... 5.81
Finished writing 2001, waiting ... 5.92
Finished writing 2002, waiting ... 5.91
Finished writing 2003, waiting ... 5.49
Finished writing 2004, waiting ... 4.87
Finished writing 2005, waiting ... 5.72
Finished writing 2006, waiting ... 4.89
Finished writing 2007, waiting ... 5.78
Finished writing 2008, waiting ... 4.53
Finished writing 2009, waiting ... 4.83
Finished writing 2010, waiting ... 5.66
Finished writing 2011, waiting ... 5.73
Finished writing 2012, waiting ... 4.08
Finished writing 2013, waiting ... 5.87
Finished writing 2014, waiting ... 5.69
F

In [26]:
all = []

for year in years:
    with open(f'webscraping/team/rank/{year}.html', encoding="utf-8") as file:
        page = file.read()
    soup = BeautifulSoup(page, 'html.parser')
    table = soup.find('table',  {'id' : 'ratings'})
    soup.find('tr', class_ = 'over_header').decompose()
    df = pd.read_html(str(table))[0]  
    df['Year'] = year
    
    print(f'Finished scraping {year}')
    all.append(df)    

Finished scraping 1990
Finished scraping 1991
Finished scraping 1992
Finished scraping 1993
Finished scraping 1994
Finished scraping 1995
Finished scraping 1996
Finished scraping 1997
Finished scraping 1998
Finished scraping 1999
Finished scraping 2000
Finished scraping 2001
Finished scraping 2002
Finished scraping 2003
Finished scraping 2004
Finished scraping 2005
Finished scraping 2006
Finished scraping 2007
Finished scraping 2008
Finished scraping 2009
Finished scraping 2010
Finished scraping 2011
Finished scraping 2012
Finished scraping 2013
Finished scraping 2014
Finished scraping 2015
Finished scraping 2016
Finished scraping 2017
Finished scraping 2018
Finished scraping 2019
Finished scraping 2020
Finished scraping 2021
Finished scraping 2022


In [27]:
team_rank = pd.concat(all)
team_rank.to_csv('../data/team_rank.csv', index=False)
print(team_rank.shape)
print(f'{team_rank.Year.min()}-{team_rank.Year.max()}')

(966, 16)
1990-2022


## VII. Team Standings

In [28]:
for year in years:
    url = f'https://www.basketball-reference.com/leagues/NBA_{year+1}_standings.html'
    driver.get(url)
    driver.execute_script('window.scrollTo(1,10000)') # Assures entire page is saved for scraping
    html = driver.page_source

    with open(f'webscraping/team/seed/{year}.html', "w+", encoding="utf-8") as file:
        file.write(html)
    lag = np.random.uniform(4,6)
    print(f'Finished writing {year}, waiting ... {round(lag,2)}')
    time.sleep(lag)

Finished writing 1990, waiting ... 4.98
Finished writing 1991, waiting ... 5.33
Finished writing 1992, waiting ... 5.41
Finished writing 1993, waiting ... 4.46
Finished writing 1994, waiting ... 5.07
Finished writing 1995, waiting ... 4.5
Finished writing 1996, waiting ... 5.8
Finished writing 1997, waiting ... 5.56
Finished writing 1998, waiting ... 5.85
Finished writing 1999, waiting ... 4.83
Finished writing 2000, waiting ... 5.92
Finished writing 2001, waiting ... 4.3
Finished writing 2002, waiting ... 5.31
Finished writing 2003, waiting ... 4.42
Finished writing 2004, waiting ... 4.36
Finished writing 2005, waiting ... 5.25
Finished writing 2006, waiting ... 5.67
Finished writing 2007, waiting ... 4.19
Finished writing 2008, waiting ... 4.44
Finished writing 2009, waiting ... 5.52
Finished writing 2010, waiting ... 5.53
Finished writing 2011, waiting ... 5.77
Finished writing 2012, waiting ... 5.32
Finished writing 2013, waiting ... 4.6
Finished writing 2014, waiting ... 4.32
Fini

In [29]:
all = []

for year in years:  
    with open(f'webscraping/team/seed/{year}.html', encoding="utf-8") as file:
        page = file.read()
    soup = BeautifulSoup(page, 'html.parser')
    table = soup.find('table', {'id' : 'expanded_standings'})
    #soup.find('tr', class_ = 'over_header').decompose()
                  
    df = pd.read_html(str(table), header=1)[0] 
    df['Year'] = year   
    
    print(f'Finished scraping {year}')
    all.append(df)    

Finished scraping 1990
Finished scraping 1991
Finished scraping 1992
Finished scraping 1993
Finished scraping 1994
Finished scraping 1995
Finished scraping 1996
Finished scraping 1997
Finished scraping 1998
Finished scraping 1999
Finished scraping 2000
Finished scraping 2001
Finished scraping 2002
Finished scraping 2003
Finished scraping 2004
Finished scraping 2005
Finished scraping 2006
Finished scraping 2007
Finished scraping 2008
Finished scraping 2009
Finished scraping 2010
Finished scraping 2011
Finished scraping 2012
Finished scraping 2013
Finished scraping 2014
Finished scraping 2015
Finished scraping 2016
Finished scraping 2017
Finished scraping 2018
Finished scraping 2019
Finished scraping 2020
Finished scraping 2021
Finished scraping 2022


In [30]:
seed = pd.concat(all)
seed.to_csv('../data/team_seed.csv', index=False)
print(seed.shape)
print(f'{seed.Year.min()}-{seed.Year.max()}')

(1004, 29)
1990-2022


## VIII. All-Star Appearance

In [31]:
url = 'https://en.wikipedia.org/wiki/List_of_NBA_All-Stars'
res = requests.get(url)
if res.status_code >= 200:
    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find('table', class_="wikitable sortable")
    
    as_appearance = pd.read_html(str(table))[0]

as_appearance.to_csv('../data/all_star_appearances.csv', index=False)    

## **4. Misc Data**

## IV. NBA Draft Year

In [32]:
url = 'https://www.nba.com/stats/draft/history' #if there were escape characters, could have specified r'https://www.nba.com/stats/draft/history'
driver.get(url)

In [33]:
element = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select")
select = Select(element)

In [34]:
# 'All' is first element, so we will need to index 0
select.select_by_index(0)

In [35]:
src = driver.page_source
parser = BeautifulSoup(src, 'lxml')
table = parser.find('table', class_= 'Crom_table__p1iZz')
draft_year = pd.read_html(str(table))[0]

In [36]:
draft_year

,Player,Team,Affiliation,Year,Round Number,Round Pick,Overall Pick
0,Victor Wembanyama,San Antonio Spurs,Metropolitans 92 (France),2023,1,1,1
1,Brandon Miller,Charlotte Hornets,Alabama,2023,1,2,2
2,Scoot Henderson,Portland Trail Blazers,Ignite (G League),2023,1,3,3
3,Amen Thompson,Houston Rockets,Overtime Elite,2023,1,4,4
4,Ausar Thompson,Detroit Pistons,Overtime Elite,2023,1,5,5
...,...,...,...,...,...,...,...
8252,Carl Reichert,New York Knicks,Findlay,1947,0,0,0
8253,Dan Miller,New York Knicks,Saint Louis,1947,0,0,0
8254,Ron Livingston,New York Knicks,St. Mary's (CA),1947,0,0,0
8255,Garland Head,New York Knicks,Texas Tech,1947,0,0,0


In [37]:
draft_year.to_csv('../data/draft_year.csv', index=False)

#### Special Thanks to These Sources

In [38]:
# Webscraping Tutorials Used To Complete

# https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a
# https://towardsdatascience.com/intro-to-web-scrapping-how-can-we-easily-seek-the-data-we-want-d9f3d9359246
# https://alexsl.medium.com/installing-selenium-webdriver-with-python-and-pycharm-from-scratch-on-windows-e4c713043882
# https://betterprogramming.pub/nba-web-scraping-with-python-22c76cfd1d4f
# https://www.geeksforgeeks.org/find_element_by_xpath-driver-method-selenium-python/?ref=rp
# https://towardsdatascience.com/how-scraping-nba-stats-is-cooler-than-michael-jordan-49d7562ce3ef

# https://www.youtube.com/watch?v=JGQGd-oa0l4&t=735s
# https://www.youtube.com/watch?v=fAW6AxMHego
# https://www.youtube.com/watch?v=lRT6YK0QcLw
# https://www.youtube.com/watch?v=nHtlRlWmTV4&t=342s